# Morse Code Screen Demo

This notebook converts text to Morse code and renders transmission on a visual screen.

- Type a message
- Set sending speed in words per minute (WPM), default **5**
- Click **Send Morse** to transmit


In [5]:
import time
import ipywidgets as widgets
from IPython.display import display

MORSE_CODE = {
    'A': '.-', 'B': '-...', 'C': '-.-.', 'D': '-..', 'E': '.', 'F': '..-.',
    'G': '--.', 'H': '....', 'I': '..', 'J': '.---', 'K': '-.-', 'L': '.-..',
    'M': '--', 'N': '-.', 'O': '---', 'P': '.--.', 'Q': '--.-', 'R': '.-.',
    'S': '...', 'T': '-', 'U': '..-', 'V': '...-', 'W': '.--', 'X': '-..-',
    'Y': '-.--', 'Z': '--..',
    '0': '-----', '1': '.----', '2': '..---', '3': '...--', '4': '....-',
    '5': '.....', '6': '-....', '7': '--...', '8': '---..', '9': '----.',
    '.': '.-.-.-', ',': '--..--', '?': '..--..', '!': '-.-.--', '/': '-..-.',
    '(': '-.--.', ')': '-.--.-', '&': '.-...', ':': '---...', ';': '-.-.-.',
    '=': '-...-', '+': '.-.-.', '-': '-....-', '_': '..--.-', '"': '.-..-.',
    '$': '...-..-', '@': '.--.-.'
}

def text_to_morse(text: str) -> str:
    tokens = []
    for ch in text.upper():
        if ch == ' ':
            tokens.append('/')
        elif ch in MORSE_CODE:
            tokens.append(MORSE_CODE[ch])
    return ' '.join(tokens)

def build_timeline(text: str):
    """Return list of (state, units, label). state=True means signal ON."""
    timeline = []
    message = text.upper()

    for i, ch in enumerate(message):
        if ch == ' ':
            if timeline and timeline[-1][0] is False:
                prev_state, prev_units, prev_label = timeline[-1]
                timeline[-1] = (prev_state, max(prev_units, 7), prev_label)
            else:
                timeline.append((False, 7, 'word gap'))
            continue

        code = MORSE_CODE.get(ch)
        if not code:
            continue

        for j, sym in enumerate(code):
            if sym == '.':
                timeline.append((True, 1, f'{ch}: dot'))
            else:
                timeline.append((True, 3, f'{ch}: dash'))

            if j < len(code) - 1:
                timeline.append((False, 1, 'symbol gap'))

        if i < len(message) - 1 and message[i + 1] != ' ':
            timeline.append((False, 3, 'letter gap'))

    return timeline


In [6]:
message_input = widgets.Text(
    value='HELLO WORLD',
    description='Message:',
    layout=widgets.Layout(width='70%')
)

wpm_input = widgets.IntSlider(
    value=5,
    min=5,
    max=40,
    step=1,
    description='WPM:',
    continuous_update=False
)

send_button = widgets.Button(
    description='Send Morse',
    button_style='primary'
)

morse_preview = widgets.HTML(value='')
status = widgets.HTML(value='<b>Status:</b> idle')
screen = widgets.HTML(value='')


def render_screen(on: bool):
    bg = '#FDFDFD' if on else '#111111'
    fg = '#111111' if on else '#FDFDFD'
    label = 'SIGNAL ON' if on else 'SIGNAL OFF'
    screen.value = (
        f"<div style='width:100%;height:580px;border:2px solid #222;background:{bg};"
        f"display:flex;align-items:center;justify-content:center;"
        f"font-family:monospace;font-size:20px;color:{fg};'>{label}</div>"
    )


def send_morse(_):
    text = message_input.value.strip()
    if not text:
        status.value = '<b>Status:</b> please enter a message'
        return

    dot_seconds = 1.2 / int(wpm_input.value)
    morse = text_to_morse(text)
    timeline = build_timeline(text)

    morse_preview.value = f'<b>Morse:</b> <code>{morse}</code>'
    status.value = f'<b>Status:</b> sending at {wpm_input.value} WPM (dot={dot_seconds:.3f}s)'

    for state, units, label in timeline:
        render_screen(state)
        status.value = f'<b>Status:</b> {label} | {wpm_input.value} WPM'
        time.sleep(units * dot_seconds)

    render_screen(False)
    status.value = '<b>Status:</b> complete'


send_button.on_click(send_morse)
render_screen(False)

display(widgets.VBox([
    message_input,
    wpm_input,
    send_button,
    morse_preview,
    status,
    screen
]))
